In [1]:
%%html

<!-- Just run this cell to provide style to the notebook-->
<!-- This is a code to add style to the notebook, it is based on a .css on GitHub http://bit.ly/1Bf5Hft -->

<style>

html {
  font-size: 62.5% !important; }
body {
  font-size: 1.5em !important; /* currently ems cause chrome bug misinterpreting rems on body element */
  line-height: 1.6 !important;
  font-weight: 400 !important;
  font-family: "Raleway", "HelveticaNeue", "Helvetica Neue", Helvetica, Arial, sans-serif !important;
  color: #222 !important; }

div{ border-radius: 0px !important;  }
div.CodeMirror-sizer{ background: rgb(244, 244, 248) !important; }
div.input_area{ background: rgb(244, 244, 248) !important; }

div.out_prompt_overlay:hover{ background: rgb(244, 244, 248) !important; }
div.input_prompt:hover{ background: rgb(244, 244, 248) !important; }

h1, h2, h3, h4, h5, h6 {
  color: #333 !important;
  margin-top: 0 !important;
  margin-bottom: 2rem !important;
  font-weight: 300 !important;
    text-decoration: underline;
}
h1 { font-size: 4.0rem !important; line-height: 1.2 !important;  letter-spacing: -.1rem !important;}
h2 { font-size: 3.6rem !important; line-height: 1.25 !important; letter-spacing: -.1rem !important; }
h3 { font-size: 3.0rem !important; line-height: 1.3 !important;  letter-spacing: -.1rem !important; }
h4 { font-size: 2.4rem !important; line-height: 1.35 !important; letter-spacing: -.08rem !important; }
h5 { font-size: 1.8rem !important; line-height: 1.5 !important;  letter-spacing: -.05rem !important; }
h6 { font-size: 1.5rem !important; line-height: 1.6 !important;  letter-spacing: 0 !important; }
    
@media (min-width: 550px) {
  h1 { font-size: 5.0rem !important; }
  h2 { font-size: 4.2rem !important; }
  h3 { font-size: 3.6rem !important; }
  h4 { font-size: 3.0rem !important; }
  h5 { font-size: 2.4rem !important; }
  h6 { font-size: 1.5rem !important; }
}

p {
    margin-top: 0 !important;
    margin-bottom: 1rem !important;
    text-align: justify;
    text-justify: inter-word;
    line-height: 1.5 !important;
    font-size: 1.2em !important;
    font-family: "Raleway", "HelveticaNeue", "Helvetica Neue", Helvetica, Arial, sans-serif !important;
}
  
a {
  color: #1EAEDB !important; }
a:hover {
  color: #0FA0CE !important; }
  
code {
  padding: .2rem .5rem !important;
  margin: 0 .2rem !important;
  font-size: 90% !important;
  white-space: nowrap !important;
  background: #F1F1F1 !important;
  border: 1px solid #E1E1E1 !important;
  border-radius: 4px !important; }
pre > code {
  display: block !important;
  padding: 1rem 1.5rem !important;
  white-space: pre !important; }
  
button{ border-radius: 0px !important; }
.navbar-inner{ background-image: none !important;  }
select, textarea{ border-radius: 0px !important; }
    
#Top_Header {
    background-image: url(https://lh3.googleusercontent.com/-shldIGCRFdbscyKdAvFOATYt3wRk3SWzxQ-OSLQvvGQQEekDY2nrNYhcTl9nB3gcx_t=w300), url(https://lh3.googleusercontent.com/-shldIGCRFdbscyKdAvFOATYt3wRk3SWzxQ-OSLQvvGQQEekDY2nrNYhcTl9nB3gcx_t=w300);
    background-size: contain;
    background-repeat: no-repeat;
    background-position: left, right;
}

.output {
    display: flex;
    align-items: center;
    text-align: center;
}
    
</style>
    
<script> 
$( document ).ready(function () {
    $("div#notebook-container").children().first().hide();
});
</script>

In [ ]:
%%javascript
//Execute this code to show the styling of the document
$("div#notebook-container").children().first().show();

<div id="Top_Header">
    <center>
        <h1>Dog Breed Identification</h1>
        <h3>Deep Learning: Transfer Learning</h3>
        <h6>Andreu Masdeu, TFG 2018</h6>
    </center>
</div>

<div id="Introduction">
    <h2>Introduction</h2>
    <p>This notebook is an approach to the <i>Kaggle</i> competition <strong><a href="https://www.kaggle.com/c/dog-breed-identification">Dog Breed Identification</a></strong> using <i>Convolutional Neural Networks</i> and <i> Transfer Learning</i>.</p>
    <p><i> Transfer Learning</i> is an excellence approach for problems where there isn't a large amount of data. Using pre-trained models helps us increase our accuracy, since these models are trained on datasets with millions of images. Adding our own fully connected layers to adapt these models to this particular problem is the final step for this procedure. </p>
</div>

<h1> Importing </h1>

In [ ]:
import pandas as pd
import numpy as np

import keras
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Conv2D, MaxPooling2D, Flatten, Dropout, LeakyReLU, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xcep_process

import tensorflow as tf

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode()
import matplotlib.pyplot as plt
from IPython.display import clear_output


from sklearn.model_selection import train_test_split
import cv2
from tqdm import tqdm
import time
import random
import os

<h1> Auxiliar functions </h1>

<p> The following cell includes some useful functions. Get_pixels transforms a picture into an array of numbers with the desired dimension and color channel encoding. Normalize, as its name suggests, normalizes those arrays into an interval [0,1]. Finally, show_image creates a window showing the picture given and it is useful to see differences between normalized and normal pictures. </p>

In [ ]:
def get_pixels(dim, path ):
    image = cv2.imread(path, 1)
    resized = cv2.resize(image, (dim, dim))
    return cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)

def show_image(img, dim=224):
    #img = img.reshape((dim,dim,3))
    plt.imshow(img, cmap='gray')
    plt.show()

def normalize(img):
    img  = (img-img.min())/(img.max()-img.min())
    return np.array(img, dtype= 'float32')

# Paths and labels

Labels for each photo are contained in a csv file. The following lines store in a pandas dataframe the root path and label for each picture. Finally we represent in a plot the dataset distribution for each breed.

In [ ]:
DIM = int(input("Write the number of pixels desired "))

In [ ]:
trainfolder = "train/train/"
paths_labels = pd.read_csv('labels.csv/labels.csv')
paths_labels['image_path'] = paths_labels.apply(lambda x: (trainfolder + x["id"] + ".jpg" ), axis=1)

In [ ]:
paths = paths_labels['image_path'].values

In [ ]:
lst = []
for path in tqdm(paths_labels['image_path']):
    lst.append(normalize(get_pixels(DIM,path)))
    

In [ ]:
labels = pd.get_dummies(paths_labels, columns=['breed'])

In [ ]:
data = np.array(lst)
del lst

In [ ]:
dropped = []
for i in range(0,2):
    dropped.append(labels.columns[i])
    
labels = labels.drop(dropped, axis=1, inplace=False)

In [ ]:
dic = {}
num_classes = len(labels.columns)
for i in range(0, num_classes-1):
    l = list(labels.columns[i])
    l[0:6] = []
    elem2 = "".join(l)
    dic[labels.columns[i]] = elem2
    
labels = labels.rename(dic, axis = "columns")

In [ ]:
dogs_list = labels.columns

In [ ]:
labels = labels.values

In [ ]:
num_for_breed = [0 for x in dogs_list]
for label in labels_test:
    num_for_breed[label.argmax()] += 1

In [ ]:
trace0 = go.Bar(
    x=dogs_list,
    y=num_for_breed,
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6
)

data_plot = [trace0]
layout = go.Layout(
    title='Breed distribution',
)

fig = go.Figure(data=data_plot, layout=layout)
py.iplot(fig, filename='text-hover-bar')

# Dataset splitting

The following cells create three random splits on the dataset. 

In [ ]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.1, 
                                                                    shuffle = True, random_state = 32)

In [ ]:
data_train2, data_test2, labels_train2, labels_test2 = train_test_split(data, labels, test_size=0.1, 
                                                                        shuffle = True, random_state = 34)

In [ ]:
data_train3, data_test3, labels_train3, labels_test3 = train_test_split(data, labels, test_size=0.1,
                                                                        shuffle = True, random_state=45)

<h1> Neural Network architecture </h1>

<p>In Keras, pre-trained models are stored in <i>keras.applications</i> and are treated as layers when designing a network. As we call <i>model.add(Convolution(...)</i>, we would call <i>model.add(Xception(...))</i>. Paremeters for that method are: weights, include_top and input_shape. </p>
    
<p> We can decide whether to use the pretrained weights by calling <i>weights = 'imagenet' </i>  or just stick with the structure and randomly initialize them. Include_top makes reference to include the fully connected layers in the model or not. Usually, in transfer learning, these layers are not included because the output layer differs and we are predicting new classes. Finally, input_shape is the dimension of your images in the classic format [height, width, channels]. There are default dimensions in which the models were trained, but any dimension can be used. The layer would automatically infer changes on the convolutions, poolings etc...   </p>

<p> We opted for using default dimensions for each model. First of all, we define a function that returns the model we will use. Once we get the model, since the output shape is a 3D tensor we flatten it. Then, an undefined number of fully connected layers are added, each one with dropout, batch normalization and leaky relu as activation function. The output layer is the final one, and since this is classification problem we use softmax. Finally, we compile it, and we use categorical cross entropy as our loss function since we are using the softmax activation. </p>

# Freezing weights

In this section, we train the frozen model. The first cells extract the high-level features from the images. Then we define the feedforward network in the function freeze_model. Finally, we train a copy of that model for each dataset split.

In [ ]:
feature_extractor = Xception(weights='imagenet', include_top=False, input_shape=(DIM, DIM, 3), pooling = 'avg')

In [ ]:
feature_extractor.summary()

In [ ]:
feature_vectors = []
feature_vectors_val = []
for elem in data_train:
    feature_vectors.append(feature_extractor.predict(elem.reshape(1, 299, 299, 3)).reshape(2048))
    
for elem in data_test:
    feature_vectors_val.append(feature_extractor.predict(elem.reshape(1, 299, 299, 3)).reshape(2048))


In [ ]:
feature_vectors2 = []
feature_vectors_val2 = []
for elem in data_train2:
    feature_vectors2.append(feature_extractor.predict(elem.reshape(1, 299, 299, 3)).reshape(2048))
    
for elem in data_test2:
    feature_vectors_val2.append(feature_extractor.predict(elem.reshape(1, 299, 299, 3)).reshape(2048))

In [ ]:
feature_vectors3 = []
feature_vectors_val3 = []
for elem in data_train3:
    feature_vectors3.append(feature_extractor.predict(elem.reshape(1, 299, 299, 3)).reshape(2048))
    
for elem in data_test3:
    feature_vectors_val3.append(feature_extractor.predict(elem.reshape(1, 299, 299, 3)).reshape(2048))

In [ ]:
def freeze_model( nhid = 3, drop = 0, alpha = 0.05):
    model = Sequential()
    
    if nhid != 0:
        model.add(Dense(int(512), input_shape = (2048,)))
        model.add(LeakyReLU(alpha = alpha))
        model.add(BatchNormalization())
        model.add(Dropout(drop))
    
    for i in range(0,nhid-1):
        model.add(Dense(int(216/(2 ** i))))
        model.add(LeakyReLU(alpha = alpha))
        model.add(BatchNormalization())
        model.add(Dropout(drop))
        
    if nhid == 0: model.add(Dense(120, activation = 'softmax', input_shape=(2048,)))
    else: model.add(Dense(120, activation = 'softmax'))
        
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
model_freeze = freeze_model(nhid =0 , drop = 0.2)

In [ ]:
model_freeze.summary()

In [ ]:
history_freeze = model_freeze.fit(np.array(feature_vectors), labels_train, batch_size = 16, epochs = 16, callbacks = [], 
                    validation_data = (np.array(feature_vectors_val), labels_test), verbose = 2)

In [ ]:
model_freeze2 = freeze_model(nhid =0 , drop = 0.2)
history_freeze2 = model_freeze2.fit(np.array(feature_vectors2), labels_train2, batch_size = 16, epochs = 16, callbacks = [], 
                    validation_data = (np.array(feature_vectors_val2), labels_test2), verbose = 2)

In [ ]:
model_freeze3 = freeze_model(nhid =0 , drop = 0.2)
history_freeze3 = model_freeze3.fit(np.array(feature_vectors3), labels_train3, batch_size = 16, epochs = 16, callbacks = [], 
                    validation_data = (np.array(feature_vectors_val3), labels_test3), verbose = 2)

### Using data augmentation

In this section, we train the frozen model using data augmentation. The first cell creates the data augmentation generator using the keras function ImageDataGenerator. We set values for rotation, shift, zoom and flip. Then we define the frozen model including Xception as one layer more. But we set the Xception layer to a non-trainable state to keep the weights frozen. Then we train three samples of this model using the three splits and with the method fit_generator, as we use the python generator created for artificially augmenting the number of images.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

datagen.fit(data_train)

In [ ]:
def freeze_gen_model( nhid = 3, drop = 0, alpha = 0.05):
    model = Sequential()
    
    model.add(Xception(weights='imagenet', include_top=False, input_shape=(DIM, DIM, 3), pooling = 'avg'))
    
    for i in range(0,nhid):
        model.add(Dense(int(512/(2 ** i))))
        model.add(LeakyReLU(alpha = alpha))
        model.add(BatchNormalization())
        model.add(Dropout(drop))
        
    model.add(Dense(120, activation = 'softmax'))
    
    model.layers[0].trainable = False
        
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
model_freeze_gen = freeze_gen_model(nhid = 0, drop = 0.0)

In [ ]:
model_freeze_gen.summary()

In [ ]:
history_freeze_gen = model_freeze_gen.fit_generator(datagen.flow(data_train, labels_train, batch_size = 16), epochs = 16,
                                       steps_per_epoch = data_train.shape[0]//16,
                                       validation_data = (data_test, labels_test), verbose = 2)

In [ ]:
model_freeze_gen2 = freeze_gen_model(nhid = 0, drop = 0.0)
history_freeze_gen2 = model_freeze_gen2.fit_generator(datagen.flow(data_train2, labels_train2, batch_size = 16), epochs = 16,
                                       steps_per_epoch = data_train.shape[0]//16,
                                       validation_data = (data_test2, labels_test2), verbose = 2)

In [ ]:
model_freeze_gen3 = freeze_gen_model(nhid = 0, drop = 0.0)
history_freeze_gen3 = model_freeze_gen3.fit_generator(datagen.flow(data_train3, labels_train3, batch_size = 16), epochs = 16,
                                       steps_per_epoch = data_train.shape[0]//16,
                                       validation_data = (data_test3, labels_test3), verbose = 2)

<h1> Fine-tuning </h1>

In this section, we train the fine-tuning model. We define the fine-tuning model including Xception as one layer more. Then we train three samples of this model using the three splits and with the method fit_generator, as we use the python generator created for artificially augmenting the number of images.

In [ ]:
def fine_model( nhid = 3, drop = 0, alpha = 0.05):
    model = Sequential()
    
    model.add(Xception(weights='imagenet', include_top=False, input_shape=(DIM, DIM, 3), pooling = 'avg'))
    
    for i in range(0,nhid):
        model.add(Dense(int(512/(2 ** i))))
        model.add(LeakyReLU(alpha = alpha))
        model.add(BatchNormalization())
        model.add(Dropout(drop))
        
    model.add(Dense(120, activation = 'softmax'))
        
    adam_opt = keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
model_fine = fine_model(nhid = 0, drop = 0.0)

In [ ]:
model_fine.summary()

In [ ]:
history_fine = model_fine.fit(data_train, labels_train, batch_size = 16, epochs = 16, callbacks = [], 
                    validation_data = (data_test, labels_test), verbose = 2)

In [ ]:
model_fine2 = fine_model(nhid = 0, drop = 0.0)
history_fine2 = model_fine2.fit(data_train2, labels_train2, batch_size = 16, epochs = 16, callbacks = [], 
                    validation_data = (data_test2, labels_test2), verbose = 2)

In [ ]:
model_fine3 = fine_model(nhid = 0, drop = 0.0)
history_fine3 = model_fine3.fit(data_train3, labels_train3, batch_size = 16, epochs = 16, callbacks = [], 
                    validation_data = (data_test3, labels_test3), verbose = 2)

# Data augmentation

In this section, we train the fine-tuning model using data augmentation. The first cell creates the data augmentation generator using the keras function ImageDataGenerator. We set values for rotation, shift, zoom and flip. Then we define the fine-tuning model including Xception as one layer more. Then we train three samples of this model using the three splits and with the method fit_generator, as we use the python generator created for artificially augmenting the number of images.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

datagen.fit(data_train3)

In [ ]:
model_gen = fine_model(nhid = 0, drop = 0.0)

In [ ]:
history_fine_gen = model_gen.fit_generator(datagen.flow(data_train, labels_train, batch_size = 16), epochs = 16,
                                       steps_per_epoch = data_train.shape[0]//16,
                                       validation_data = (data_test, labels_test), verbose = 2)

In [ ]:
model_gen2 = fine_model(nhid = 0, drop = 0.0)
history_fine_gen2 = model_gen2.fit_generator(datagen.flow(data_train2, labels_train2, batch_size = 16), epochs = 16,
                                       steps_per_epoch = data_train.shape[0]//16,
                                     jg  validation_data = (data_test2, labels_test2), verbose = 2)

In [ ]:
model_gen3 = fine_model(nhid = 0, drop = 0.0)
history_fine_gen3 = model_gen3.fit_generator(datagen.flow(data_train3, labels_train3, batch_size = 16), epochs = 16,
                                       steps_per_epoch = data_train3.shape[0]//16,
                                       validation_data = (data_test3, labels_test3), verbose = 2)

<h1> Plotting </h1>

As usual, a useful way of checking our model's performance is by plotting accuracy and loss. We'll do it for the 4 approaches in the first split.

### Freeze

In [ ]:
y_axis1 = history_freeze.history['acc']
y_axis2 = history_freeze.history['val_acc']
y_axis3 = history_freeze.history['loss']
y_axis4 = history_freeze.history['val_loss']
x_axis = list(range(1,17))

In [ ]:
trace1 = go.Scatter(x = x_axis, y = y_axis1, mode = 'lines+markers', name = 'Training accuracy')

trace2 = go.Scatter(x = x_axis, y = y_axis2, mode = 'lines+markers', name = 'Validation accuracy')

data1 = [trace1, trace2]

py.iplot(data1, filename='basic-line')

In [ ]:
trace2 = go.Scatter(x = x_axis, y = y_axis3, mode = 'lines+markers', name = 'Training loss')

trace3 = go.Scatter(x = x_axis, y = y_axis4, mode = 'lines+markers', name = 'Validation loss')

data2 = [trace2, trace3]

py.iplot(data2, filename='basic-line')

### Freeze using data augmentation

In [ ]:
y_axis1 = history_freeze_gen.history['acc']
y_axis2 = history_freeze_gen.history['val_acc']
y_axis3 = history_freeze_gen.history['loss']
y_axis4 = history_freeze_gen.history['val_loss']
x_axis = list(range(1,17))

In [ ]:
trace1 = go.Scatter(x = x_axis, y = y_axis1, mode = 'lines+markers', name = 'Training accuracy')

trace2 = go.Scatter(x = x_axis, y = y_axis2, mode = 'lines+markers', name = 'Validation accuracy')


data1 = [trace1, trace2]

py.iplot(data1, filename='basic-line')

In [ ]:
trace2 = go.Scatter(x = x_axis, y = y_axis3, mode = 'lines+markers', name = 'Training loss')

trace3 = go.Scatter(x = x_axis, y = y_axis4, mode = 'lines+markers', name = 'Validation loss')

data2 = [trace2, trace3]

py.iplot(data2, filename='basic-line')

### Fine-tuning

In [ ]:
y_axis1 = history_fine.history['acc']
y_axis2 = history_fine.history['val_acc']
y_axis3 = history_fine.history['loss']
y_axis4 = history_fine.history['val_loss']
x_axis = list(range(1,17))

In [ ]:
trace1 = go.Scatter(x = x_axis, y = y_axis1, mode = 'lines+markers', name = 'Training accuracy')

trace2 = go.Scatter(x = x_axis, y = y_axis2, mode = 'lines+markers', name = 'Validation accuracy')

data1 = [trace1, trace2]

py.iplot(data1, filename='basic-line')

In [ ]:
trace2 = go.Scatter(x = x_axis, y = y_axis3, mode = 'lines+markers', name = 'Training loss')

trace3 = go.Scatter(x = x_axis, y = y_axis4, mode = 'lines+markers', name = 'Validation loss')

data2 = [trace2, trace3]

py.iplot(data2, filename='basic-line')

### Fine-tuning with data augmentation

In [ ]:
y_axis1 = history_fine_gen3.history['acc']
y_axis2 = history_fine_gen3.history['val_acc']
y_axis3 = history_fine_gen3.history['loss']
y_axis4 = history_fine_gen3.history['val_loss']
x_axis = list(range(1,16))

In [ ]:
trace1 = go.Scatter(x = x_axis, y = y_axis1, mode = 'lines+markers', name = 'Training accuracy')

trace2 = go.Scatter(x = x_axis, y = y_axis2, mode = 'lines+markers', name = 'Validation accuracy')

data1 = [trace1, trace2]

py.iplot(data1, filename='basic-line')

In [ ]:
trace2 = go.Scatter(x = x_axis, y = y_axis3, mode = 'lines+markers', name = 'Training loss')

trace3 = go.Scatter(x = x_axis, y = y_axis4, mode = 'lines+markers', name = 'Validation loss')

data2 = [trace2, trace3]

py.iplot(data2, filename='basic-line')

# Saving and loading

Since training takes so long, it is wise to save the model if results were interesting. Also, we need the weights and the architecture in a file for uploading them into the AWS Lambda function.

In [ ]:
import os
from keras.models import model_from_json

In [ ]:
list_models = [model_freeze,  model_freeze_gen, 
               model_fine,  model_gen, ]

In [ ]:
for i, model1 in enumerate(list_models):
    model_json = model1.model.to_json()
    with open("model_architecture{}.json".format(i), "w") as json_file:
        json_file.write(model_json)
    model1.model.save_weights("model_weights{}.h5".format(i))

In [ ]:
# load json and create model
json_file = open('model_architecture3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model3 = model_from_json(loaded_model_json)

In [ ]:
# load weights into new model

model3.load_weights("model_weights3.h5")

# Validation

The following cells are used for validating the model. First of all, we define the function get_dog which returns the name of the dog given the probability vector. Then we define the function, check_some_dogs which let us visualize the model on real time.

In [ ]:
from random import randint

def get_dog(v):
    
    return dogs_list[v.argmax()]

def check_some_dogs(model, paths, labels):
    
    errors = 0
    for i in range(0, len(paths)):
                
        img = normalize(get_pixels(299, paths[i]))
        
        plt.imshow(img)
        plt.show()
        pred = model.predict(img.reshape((1,299,299,3)))
        dog = get_dog(pred)
        print("Correct label: ", get_dog(labels[i]), "  ")
        print("Predicted label: ", dog , "  ")
        print("Probability: ", pred.max()*100, '% ')
        
        if dog != get_dog(labels[i]): errors = errors + 1
        
        time.sleep(2)
        if i != len(paths)-1: clear_output()
        
    print("Checked {} images".format(len(paths)), "  ")
    print("Estimated accuracy: ", 100-100*errors/len(paths), "%   " )
    
    
    
        
        

In [ ]:
check_some_dogs(model, paths[500:560], labels[500:560])

### Confusion matrix

The following cells implement methods for visualizing confusion matrices and the F1-Score for each class. First of all, we import the sklearn function, confusion_matrix, which builds a confusion matrix using the predicted labels and the true labels. Then we define functions for computing, recall and precision from the confusion matrix. Once we have this functions we can define the F1-Score. Also, we defined a function for visualizating the confusion matrices, plot_conf_mat.

In [ ]:
from sklearn.metrics import confusion_matrix

def recall_fun(conf_mat):
    
    recall_list=[]
    size_mat = len(conf_mat)
    
    for i in range(0,size_mat):
        true_positive = 0
        false_negative = 0
        for j in range(0,size_mat):
            if(i==j): true_positive = conf_mat[i,j]
            else: false_negative += conf_mat[i,j]
        if true_positive+false_negative == 0: recall_k = 0
        else:
            recall_k = true_positive/(true_positive+false_negative)
        recall_list.append(recall_k)
        
    return recall_list

def precision_fun(conf_mat):
    
    precision_list=[]
    size_mat = len(conf_mat)
    
    for j in range(0,size_mat):
        true_positive = 0
        false_positive = 0
        for i in range(0,size_mat):
            if(i==j): true_positive = conf_mat[i,j]
            else: false_positive += conf_mat[i,j]
                
        if true_positive+false_positive == 0: precision_k = 0
        else:
            precision_k = true_positive/(true_positive+false_positive)
        precision_list.append(precision_k)
        
    return precision_list

def F1_score(conf_mat, mode = 1):
    recall = recall_fun(conf_mat)
    precision = precision_fun(conf_mat)
    
    size_mat = len(conf_mat)
    F1 = 0
    f1 = []
    
    for i in range(0,size_mat):
        if precision[i]+recall[i] == 0: f1_k = 0
            
        else: 
            f1_k = (2*precision[i]*recall[i]) / (precision[i]+recall[i])
            F1 += f1_k
        if mode: 
            print('F1 score for {} is {} '.format(dogs_list[i], f1_k ))
            print('')
            print('Precision: {}, Recall: {} '.format(precision[i], recall[i]))
            print('')
        f1.append(f1_k)
        
    print('Mean F1 score is {}'.format(F1/size_mat))  
    return F1/size_mat, f1

def plot_confusion_mat(model, images, labels, mode = 1):
    
    predicted_labels = []
    true_labels = []
    
    for i in range(0, len(images)):
        
        if mode: dog = get_dog(model.predict(images[i].reshape(1,299, 299, 3)))
        else: dog = get_dog(model.predict(images[i].reshape(1,2048)))
        true_labels.append(get_dog(labels[i]))
        predicted_labels.append(dog)
                          
    matrix = confusion_matrix(true_labels, predicted_labels)
    
    classes = dogs_list
    trace = go.Heatmap(z=matrix,
                    colorscale="Greens", reversescale=True)
    
    layout = go.Layout( title='Confusion Matrix', xaxis={'title':'Predicted Classes'}, yaxis={'title':'True Classes'})
    
    fig = go.Figure(data=[trace], layout=layout)
    py.iplot(fig, filename='labelled-heatmap', show_link=False)
    return matrix

### Confusion matrices and F1-Scores for the freeze approach using data augmentation and the fine-tuning approaches

In [ ]:
conf_mat_list = []

list_models = [model, model2, model3]

for model in list_models:
    
    conf_mat = plot_confusion_mat(model, data_test, labels_test)
    conf_mat_list.append(conf_mat)

In [ ]:
for conf_mat in conf_mat_list:
    F1_score(conf_mat)


In [ ]:
for conf_mat in conf_mat_list:
    mean, f1_components = F1_score(conf_mat, 0)
    trace = go.Scatter(x = dogs_list, y = f1_components, mode = 'lines+markers', name = 'F1 score per class')


    data = [trace]
    
    layout = dict(title = 'F1 Score',
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False)
             )
    fig = dict(data=data, layout=layout)
    py.iplot(fig, filename='basic-line')
    

#### Freeze approach: confusion matrix and F1-Score

In [ ]:
conf_mat = plot_confusion_mat(model_freeze, feature_vectors_val, labels_test, 0)
mean, f1_components = F1_score(conf_mat, 0)
trace = go.Scatter(x = dogs_list, y = f1_components, mode = 'lines+markers', name = 'F1 score per class')


data = [trace]
    
layout = dict(title = 'F1 Score',
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False)
             )
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='basic-line')

In [ ]:
dogs_list = y_train.columns.tolist()

print(dogs_list[model.predict(normalize(get_pixels(299, 'bond.jpg')).reshape((1,299,299,3))).argmax()])